# Pipeline Infrastructure

1. compute some scalar value for all assets
2. filter assets based off that scalar value.
3. set desired portfolio weights of filtered assets.
4. Place orders on assets to reflect desired portfolio weights

#### Classifer

A classifier is a functiono that transfroms the input of an asset and a timestamp into a 
categorical output such as taking in AAPL and 2017 and reporting back that i belongs to the 
tech sector.

#### Factor

A factor is a functiono that takes in an asset and a timestamp and returns a numerical value,
such as the 10 day moving average

Create a pipelines that returns all equities available at some timestamp
Grab everything then filter out

Then we create a factor that will compute smoe numerical value given the pricing information

A lot of these functions will be imported from built in Quantopian Pipeline libraries.


#### FIlters

Filters take in an asset and a timestamp and return a boolean

#### Screens

Screens then allow you to execture those filters in your pipeline

#### Masking And Classifiers

Masking allows us to tell our pipeline to ignore assets all together, before the factors or filters even really atake place.

We can pass in the mask parameter to both factors and filters

Classifiers take in an asset and a timestamp and return a categorical value such as a sector or an exchange.

In [1]:
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar


def initialize(context):
     
    
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe,'my_pipeline')
    
    schedule_function(my_rebalance, date_rules.week_start(),
                      time_rules.market_open(hours=1))
    
    

def my_rebalance(context,data):
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security,0)
    
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security,context.long_weight)
    
    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security,context.short_weight)
    

def compute_weights(context):
    if len(context.longs) == 0:
        long_weight = 0
    else:
        long_weight = 0.5 / len(context.longs)
    if len(context.shorts) == 0:
        short_weight = 0
    else:
        short_weight = -0.5 / len(context.shorts)

        
    
    return long_weight, short_weight

def before_trading_start(context,data):
    context.output =  pipeline_output('my_pipeline')
    print(context.output['longs'])
    # LONG
    context.longs = context.output[context.output['longs']].index.tolist()
   
        
    # SHORT
    context.shorts = context.output[context.output['shorts']].index.tolist()
    
    context.long_weight,context.short_weight = compute_weights(context)

    


def make_pipeline():
    # Universe of Q1500
    base_universe = Q1500US()
    
    # Energy Sector
    sector = morningstar.asset_classification.morningstar_sector_code.latest
    energy_sector = sector.eq(309)
    
    # Make a mask of 1500US and Energy
    base_energy = base_universe & energy_sector
    
    # Dollar volume (30 days) Grab the iniformation
    dollar_volume = AverageDollarVolume(window_length=30)
    
    # Grab the top 5% in avg dollar volume
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    # Combine the filters
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day mean close
    mean_10 = SimpleMovingAverage(inputs=[
        USEquityPricing.close
    ],window_length=10,mask=top_five_base_energy)
    
    # 30 day mean close
    mean_30 = SimpleMovingAverage(inputs=[
        USEquityPricing.close
    ],window_length=30,mask=top_five_base_energy)
    
    # Percent difference between
    percent_diff = (mean_10-mean_30) / mean_30
    
    # List of shorts
    shorts = percent_diff < 0
    
    # List of Longs
    longs = percent_diff > 0
    
    # Final mask/Filter for anything in shorts or longs
    securities_to_trade = (shorts | longs)
    
    # Return Pipeline
    return Pipeline(
        columns ={
            'longs': longs,
            'shorts': shorts,
            'percent_diff': percent_diff,
        },
        screen=securities_to_trade
    )
    

ModuleNotFoundError: No module named 'quantopian'

In [2]:
# Levarage example
"""
This is a template algorithm on Quantopian for you to adapt and fill in.
"""
import quantopian.algorithm as algo
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS


def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    set_max_leverage(1.05)
    context.amzn = sid(16841)
    context.uso = sid(28320)
    
    schedule_function(rebalance,date_rules.every_day(),time_rules.market_open())
    schedule_function(record_vars,date_rules.every_day(),time_rules.market_close())


def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation
    on pipeline can be found here:
    https://www.quantopian.com/help#pipeline-title
    """

    # Base universe set to the QTradableStocksUS
    pass

def before_trading_start(context, data):
    """
    Called every day before market open.
    """
    pass


def rebalance(context, data):
    """
    Execute orders according to our schedule_function() timing.
    """
    order_target_percent(context.amzn,0.5)
    order_target_percent(context.uso,0.5)


def record_vars(context, data):
    """
    Plot variables at the end of each day.
    """
    record(amzn_close=data.current(context.amzn,'close'))
    record(ibm_close=data.current(context.uso,'close'))
    record(Leverage = context.account.leverage)
    record(Exposure = context.account.net_leverage)


def handle_data(context, data):
    """
    Called every minute.
    """
    pass

ModuleNotFoundError: No module named 'quantopian'